In [1]:
import tensorflow_hub as hub

# SavedModels from TF Hub in TF 2

在 TF Hub 上共享预训练的模型和模型块时，推荐使用 TF2的 SavedMode 格式；下面说明怎样在 TF2 程序中使用 low-level 的`hub.load()` API 及其打包函数 `hub.KerasLayer`（通常使用`hub.KerasLayer`和`tf.keras.layers`来构建 Keras 模型或 TF2 Estimator 的`model_fn`）；这两个 API 也可以以 TF1 Hub 的格式加载模型，但会存在一些限制，详见下面的[模型兼容性](#4.-Model-compatibility-for-TF1/TF2)




## 1. Using SavedModels from TF Hub

### 1.1 Using a SavedModel in Keras
Keras 是 TensorFlow 的高级 API；通过组合 Keras 模块中的的网络层对象，可以构建得到深度学习模型；`tensorflow_hub`库提供的`hub.KerasLayer`类可以使用 SavedModel 的 URL 或文件系统路径初始化，并提供 SavedModel 中的计算以及预训练的权重；`hub.KerasLayer`中模型的权重默认为不可训练的，若需要其为可训练，可参见下面的 [Fine-Tuning](#1.3-Fine-Tuning) 章节；下面是使用预训练文本嵌入并构建文本分类器的示例：
```python
hub_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
embed = hub.KerasLayer(hub_url)
embeddings = embed(["A sentence.", "word", "http://example.com"])
model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])
```
训练并验证这样一个分类器的示例可参见 [Text classification colab](https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_text_classification.ipynb)

### 1.2 Using a SavedModel in low-level TensorFlow
`hub.load(handle)`函数包装了 TF2 的`tf.saved_model.load()`函数；调用`hub.load(handle)`会下载 SavedModel，并将其解压、返回由`tf.saved_model.load()`加载的结果，因此`hub.load()`可以处理任何有效的 SavedModel；如何使用 SavedModel 可参见 [SavedModel Guide](https://www.tensorflow.org/guide/saved_model)

### 1.3 Using a SavedModel in an Estimator
对于使用 TF 的 [Estimator](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_estimator) API 进行分布式训练的用户，可以通过在其他`tf.keras.layer`中根据`hub.KerasLayer`编写`model_fn`来使用 SavedModel；
```python
# DEPRECATED: TensorFlow 1
m = hub.Module(handle, tags={"foo", "bar"})
tensors_out_dict = m(dict(x1=..., x2=...), signature="sig", as_dict=True)
# TensorFlow 2
m = hub.load(path, tags={"foo", "bar"})
tensors_out_dict = m.signatures["sig"](x1=..., x2=...)
```
此外，若要在 Estimator 中使用 TF2 SavedModel 进行参数服务器(parameter servers)的训练，需要在`tf.Session`的配置协议中设置`experimental.share_cluster_devices_in_session`，否则可能会抛出异常`"Assigned device '/job:ps/replica:0/task:0/device:CPU:0' does not match any device."`，设置方法为
```python
session_config = tf.compat.v1.ConfigProto()
session_config.experimental.share_cluster_devices_in_session = True
run_config = tf.estimator.RunConfig(..., session_config=session_config)
estimator = tf.estimator.Estimator(..., config=run_config)
```
TF2.2 之后该选项不再是实验性的，进而代码中不含`.experimental`段；

### 1.4 SavedModel downloading and caching
使用 TF Hub 或其他能够实现其托管协议 ([hosting protocol](https://www.tensorflow.org/hub/hosting)) 的 HTTPS 服务器的 SavedModel 时，会默认下载并将其解压到本地文件系统；用户可以通过重写环境变量`TFHUB_CACHE_DIR`来改变下载及解压的临时地址，详见 [Caching](#2.-Caching-model-downloads-from-TF-Hub)

### 1.5 what to expect from the SavedModel after loading
根据 SavedModel 的内容，`obj = hub.load()`的结果可以被多种方式调用，这在 TensorFlow 的 [SavedModel Guide](https://www.tensorflow.org/guide/saved_model) 中有更详细的解释
- SavedModel 的 serving signature 需要表示为具体函数的字典，该函数可以通过`tensors_out = obj.signatures("serving_default")(**tensors_in)`调用；张量字典则以相应的输入和输出的名称作为键值，并且服从 signature 中对形状和数据类型的约束
- 所保存对象的由 [@tf.function](https://www.tensorflow.org/api_docs/python/tf/function) 修饰的方法会以`tf.function`对象的形式被恢复，这些对象可以被所有张量和非张量参数的组合所调用，而`tf.function`会一直追踪着些参数直至这些参数被保存；特别地，如果有一个具有适当跟踪的`obj.__call__`方法，则可以像调用 Python 函数一样调用`obj`本身，即`output_tensor = obj(input_tensor, training=False)`

`obj`的可重用 SavedModel 接口建立了使用的约定，这样客户端代码以及类似`hub.KerasLayer`这样的适配器就可以知道如何使用 SavedModel 了；然而一些保存的模型可能不遵循这种约定，特别是当其不打算在更大的模型中重用，而只是提供 serving signatures 时；SavedModel 中可训练变量将被重新加载为可训练变量，`tf.GradientTape`默认会追踪这些变量；在进行微调时须注意`obj.trainable_variables`是否建议只对最初可训练变量的子集进行重新训练






## 2. Creating SavedModels for TF Hub
SavedModel 是 TensorFlow 中用于已训练模型和模型片段的标准序列化格式；它将模型中训练好的权重和相关操作储存在一起，以执行相应的计算；它可以独立于创建它的代码使用，并可以在不同的 high-level 的构建模型的 API 之间重用，例如如 Keras；

### 2.1 利用 Keras 保存模型
从 TensorFlow 2开始，`tf.keras.Model.save()`和`tf.keras.models.save_model()`默认生成 SavedModel 格式，生成的 SavedModel 可以与 `hub.load()`，`hub.KerasLayer`和其他类似的 high-level API 适配器一起使用。若需要共享一个完整的 Keras 模型，只需保存时设置`include_optimizer=False`；若需要共享 Keras 模型的一部分，则需要先将这一部分声明为一个子模型，再对其进行保存，可参见对 [BERT 的模型保存](https://github.com/tensorflow/models/tree/master/official/nlp/bert)；或者在创建模型后提取其中一部分进行共享，前提是子模型与原模型的分层一致，可参见对 [ResNet 的模型保存](https://github.com/tensorflow/models/blob/master/official/vision/image_classification/resnet/tfhub_export.py)


### 2.2 Saving from low-level TensorFlow
使用这种方法前提是对 [SavedModel Guide](https://www.tensorflow.org/guide/saved_model) 足够熟悉。

如果希望提供的不仅仅是 serving signature，则需要实现 [Reusable SavedModel interface](https://www.tensorflow.org/hub/reusable_saved_models)，方式如下：
```python
class MyMulModel(tf.train.Checkpoint):
    def __init__(self, v_init):
        super(MyMulModel, self).__init__()
        self.v = tf.Variable(v_init)
        self.variables = [self.v]
        self.trainable_variables = [self.v]
        self.regularization_losses = [
            tf.function(input_signature=[])(lambda: 0.001 * self.v**2),
        ]

    @tf.function(
        input_signature=[tf.TensorSpec(shape=None, dtype=tf.float32)]
    )
    def __call__(self, inputs):
        return tf.multiply(inputs, self.v)

tf.saved_model.save(MyMulModel(2.0), "/tmp/my_model")
layer = hub.KerasLayer("/tmp/my_model")
layer.trainable = True
print(layer([10., 20.]))  # ==> [20., 40.]
print(layer.trainable_weights)  # ==> [2.]
print(layer.losses)  # ==> 0.004
```




## 3 Fine-Tuning
官方建议，在建立起一个较好的训练框架，并且在 SavedModel 发布方推荐时，再考虑进行微调；需要注意的是，微调只改变模型的连续参数，而不会改变硬编码的转换 (hard-coded transformations)，例如对文本输入的 tokenize、将记  token 映射到 embedding 矩阵中对应的条目的过程；

### 3.1 For SavedModel consumers
设置模型可微调的方式为`layer = hub.KerasLayer(..., trainable=True)`；进而将 SavedModel 中声明的可训练权重和权重正则化函数添加到 Keras 模型中，并在训练模式下运行 SavedModel 的计算；一个端到端的微调过程可参见 [image classification colab](https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb)

重新导出微调结果的过程如下所示：
```python
loaded_obj = hub.load("https://tfhub.dev/...")
hub_layer = hub.KerasLayer(loaded_obj, trainable=True, ...)
model = keras.Sequential([..., hub_layer, ...])
model.compile(...)
model.fit(...)
export_module_dir = os.path.join(os.getcwd(), "finetuned_model_export")
tf.saved_model.save(loaded_obj, export_module_dir)
```


### 3.2 [For SavedModel creators](https://www.tensorflow.org/hub/tf2_saved_model)

#  

# Caching model downloads from TF Hub
`tensorflow_hub`库目前支持两种下载模型的方式：默认情况下模型以压缩文件的形式被下载并缓存到磁盘上，或者模型也可以从移动存储读入；无论哪种方式，Python 代码中对 tensorflow_hub 函数的调用可以而且应该继续使用官方的的`tfhub.dev` URL，因为这种方式可以跨系统移植，并且可以利用文档进行导航 (navigable for documentation)

## 1. Caching of compressed downloads
当模型从`tfhub.dev`或其他 hosting site 站点下载并解压后，`tensorflow_hub`库会将其缓存在文件系统上，下载位置默认为本地临时目录；除非磁盘空间有限，而网络带宽和延迟非常好，否则在大多数环境中均推荐使用此模式；

下载地址默认为本地临时目录，但最好通过设置环境变量`TFHUB_CACHE_DIR`或传递命令行`--tfhub_cache_dir`来指定；大多数情况下，默认的缓存位置`/tmp/tfhub_modules` 或任何 `os.path.join(tempfile.gettempdir(), "tfhub_modules")`指定的位置是可以使用的；需要注意的是，设置环境变量的方式下不会有自动清理的功能；

## 2. Reading from remote storage
通过代码`os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "UNCOMPRESSED"`或将命令行`--tfhub_model_load_format`标志设置为`UNCOMPRESSED`，可以直接从移动移动存储（如 GCS, Google Cloud Storage）读入模型


## 3. Running on TPU in Colab notebooks
在 [Colaboratory](https://colab.research.google.com/) 中下载压缩模型会与 TPU 的运行时 (runtime) 发生冲突，这是由于计算工作会被委托给另一台默认情况下不能访问缓存的机器；这种情况有两种解决方案；
1. 使用 TPU 可以访问的 GCS 存储器
    最简单的解决方法便是像上面所叙述的那样读取`tensorflow_hub`的 GCS 存储器；拥有自己 GCS 存储器的用户可以通过类似`os.environ["TFHUB_CACHE_DIR"] = "gs://my-bucket/tfhub-modules-cache"`的代码，来在自己的存储器中指定一个目录
2. redirect all reads (even of large variables) through the Colab host:
```python
load_options
= tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
reloaded_model = hub.load("https://tfhub.dev/...", options=load_options)
```

#  

# Migrating from TF1 to TF2 with TensorFlow Hub

下面主要说明如何在将代码从 TF1 迁移到 TF2 时继续使用 TF Hub；更多迁移指南参见 [migration guide](https://www.tensorflow.org/guide/migrate)；

在 TF2 以及 TF 1.15 中，TF Hub 不再使用`hub.Module` API 来构建`tf.compat.v1.Graph`，而是使用`hub.KerasLayer`和其他 Keras 层来构造`tf.keras.Model`，并对 low-level 的 TensorFlow 代码中使用 underlying 的`hub.load()`方法；这两个 API 可以加载 TF2 SavedModel；以 TF1 Hub 格式加载则会存在某些限制，参见[模型兼容性](#4.-Model-compatibility-for-TF1/TF2)；此外，为能够在 TF1 以及 TF2 中的 TF1 兼容模式下运行，TF Hub 仍保留了`hub.Module` API，其只能加载 [TF1 Hub 格式](https://www.tensorflow.org/hub/tf1_hub_module)


[Loading legacy models in TF1 Hub format](https://www.tensorflow.org/hub/migration_tf2#loading_legacy_models_in_tf1_hub_format)

#  

# Model compatibility for TF1/TF2

## 1. Compatibility of the [TF1 Hub format](https://www.tensorflow.org/hub/tf1_hub_module)
操作|TF1、tf.compat.v1 环境|TF2 环境
:-:|:-:|:-:
加载模型/推断|完全支持，[Guide](https://www.tensorflow.org/hub/tf1_hub_module#using_a_module)|支持，推荐使用`hub.load`或`hub.KerasLayer`
微调|完全支持，[Guide](https://www.tensorflow.org/hub/tf1_hub_module#for_consumers)|不支持
创建模型|完全支持，[Guide](https://www.tensorflow.org/hub/tf1_hub_module#general_approach)|TF1 Hub格式是面向 TF1 的，TF2 只支持其部分功能

## 2. Compatibility of [TF2 SavedModel](#1.-SavedModels-from-TF-Hub-in-TF-2)
SavedModel 在 TF 1.15 之前的版本均不支持；

<table>
    <tr>
        <th width=120px><center>操作</th>
        <th><center>TF1.15、tf.compat.v1 环境</th>
        <th width=120px><center>TF2 环境</th>
    </tr>
    <tr>
        <td><center>加载模型/推断</td>
        <td><center>使用<code>hub.load</code>或<code>hub.KerasLayer</code></td>
        <td><center>完全支持，<a href=#1.-SavedModels-from-TF-Hub-in-TF-2>Guide</a></td>
    </tr>
    <tr align=center>
        <td><center>微调</td>
        <td><div  align=left>支持使用<code>Model.fit()</code>或使用<code>model_fn</code>包装模型的 Estimator 训练的、在<code>tf.Keras.Model</code>中使用的<code>hub.KerasLayer</code>；</td>
        <td><center>完全支持，<a href=#1.3-Fine-Tuning>Guide</a></td>
    </tr>
    <tr>
        <td><center>创建模型</td>
        <td><center>兼容模式下可以调用 TF2 的<code>tf.saved_model.save()</code>API</td>
        <td><center>完全支持，<a href=#1.2-Creating-SavedModels-for-TF-Hub>Guide</a></td>
    </tr>
</table>